In [ ]:
# from datasets import load_dataset

# ds = load_dataset("maximedb/natural_questions")

# from datasets import load_dataset

# ds = load_dataset("sentence-transformers/natural-questions")

# from datasets import load_dataset

# ds = load_dataset("JasleenSingh91/travel-questions-response")

In [54]:
import nltk
import evaluate
import numpy as np
from datasets import load_dataset
# from transformers import T5Tokenizer, DataCollatorForSeq2Seq
# from transformers import T5ForConditionalGeneration, Seq2SeqTrainingArguments, Seq2SeqTrainer
from datasets import load_dataset
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, DataCollatorForSeq2Seq, Seq2SeqTrainer, Seq2SeqTrainingArguments, DataCollatorForSeq2Seq
# import nltk

from sklearn.metrics import accuracy_score, precision_recall_fscore_support


# # Download commonly used NLTK resources
# nltk.download('punkt')
# nltk.download('stopwords')
# nltk.download('wordnet')
# nltk.download('treebank', download_dir='/mnt/data/treebank')
# nltk.data.path.append('/mnt/data/treebank')

In [55]:

# Acquire the training data from Hugging Face
ds = load_dataset("maximedb/natural_questions", trust_remote_code=True)
# ds = load_dataset("JasleenSingh91/travel-questions-response")

# ds = ds["train"].train_test_split(test_size= 0.9)
ds = ds["train"].train_test_split(test_size= 0.2)


In [56]:
ds

DatasetDict({
    train: Dataset({
        features: ['question', 'answer'],
        num_rows: 104186
    })
    test: Dataset({
        features: ['question', 'answer'],
        num_rows: 26047
    })
})

In [57]:
# Load the tokenizer, model, and data collator
MODEL_NAME = "google/flan-t5-base"

tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
model = AutoModelForSeq2SeqLM.from_pretrained(MODEL_NAME)  

# tokenizer = T5Tokenizer.from_pretrained(MODEL_NAME)
# model = T5ForConditionalGeneration.from_pretrained(MODEL_NAME)
data_collator = DataCollatorForSeq2Seq(tokenizer=tokenizer, model=model)




In [ ]:
max_len_src = 0
max_len_tgt = 0

for item in ds["train"]:
    # print(item['translation'][source_lang])
    # break
    src_ids = tokenizer.encode(item['question'])
    tgt_ids = tokenizer.encode(item['answer'])
    max_len_src = max(max_len_src, len(src_ids))
    max_len_tgt = max(max_len_tgt, len(tgt_ids))
    
print(max_len_src,max_len_tgt)

dataset_en = ""
for item in raw_datasets["train"]:
    dataset_en += " " + item['translation'][source_lang]
word_cloud = WordCloud().generate(dataset_en)
plt.imshow(word_cloud)
plt.show()

dataset_fi = ""
for item in raw_datasets["train"]:
    dataset_fi += " " + item['translation'][target_lang]
word_cloud = WordCloud().generate(dataset_fi)
plt.imshow(word_cloud)

plt.show()

38 823


In [33]:
# We prefix our tasks with "answer the question"
prefix = "Answer this question: "

# Define the preprocessing function

def preprocess_function(examples):
    """Add prefix to the sentences, tokenize the text, and set the labels"""
    # The "inputs" are the tokenized answer:
    inputs = [prefix + doc for doc in examples["question"]]
    model_inputs = tokenizer(inputs, max_length=128, truncation=True)

    # The "labels" are the tokenized outputs:
    labels = tokenizer(text_target=examples["answer"], max_length=128, truncation=True)
    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

In [34]:
# Map the preprocessing function across our dataset
tokenized_dataset = ds.map(preprocess_function, batched=True)

Map: 100%|██████████| 26047/26047 [00:01<00:00, 23795.35 examples/s]


In [35]:
# Set up Rouge score for evaluation
# nltk.download("punkt", quiet=True)
metric = evaluate.load("rouge")
meteor_score = evaluate.load("meteor")

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\pingj\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt_tab to
[nltk_data]     C:\Users\pingj\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     C:\Users\pingj\AppData\Roaming\nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


In [ ]:
import re
import numpy as np

def simple_sent_tokenize(text):
    # A naive sentence tokenizer using regex
    sentences = re.split(r'(?<=[.!?])\s+', text.strip())
    return [s for s in sentences if s]

def compute_metrics(eval_preds):
    preds, labels = eval_preds

    # Replace -100 with pad_token_id
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    preds = np.where(preds != -100, preds, tokenizer.pad_token_id)

    # Decode token IDs to text
    decoded_preds = tokenizer.batch_decode(preds, skip_special_tokens=True)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

    # Use naive sentence tokenizer instead of nltk
    decoded_preds = ["\n".join(simple_sent_tokenize(pred)) for pred in decoded_preds]
    decoded_labels = ["\n".join(simple_sent_tokenize(label)) for label in decoded_labels]

    result = metric.compute(predictions=decoded_preds, references=decoded_labels, use_stemmer=True)
    meteor_res = meteor_score.compute(predictions=decoded_preds, references=decoded_labels)
    # precision, recall, f1, _ = precision_recall_fscore_support(decoded_labels, decoded_preds, average="weighted")

    return {
            'rouge':result,
            'meteor':meteor_res,
            # 'precision':precision, 
            # 'recall': recall,
            # 'f1': f1,
            }

In [ ]:
from peft import LoraConfig, get_peft_model

lora_config = LoraConfig(
    r=10, # Rank
    lora_alpha=32,
    target_modules=["q", "v"],
    lora_dropout=0.05,
    bias="none",
    # task_type="SEQ_2_SEQ_LM" # FLAN-T5
)

peft_model = get_peft_model(model, lora_config)

In [39]:
# Global Parameters
L_RATE = 3e-4
BATCH_SIZE = 4
# PER_DEVICE_EVAL_BATCH = 2
WEIGHT_DECAY = 0.01
SAVE_TOTAL_LIM = 3
NUM_EPOCHS = 3

# Set up training arguments
training_args = Seq2SeqTrainingArguments(
    output_dir="./results",
    eval_strategy="epoch",
    save_strategy="epoch",
    learning_rate=L_RATE,
    # per_device_train_batch_size=BATCH_SIZE,
    # per_device_eval_batch_size=PER_DEVICE_EVAL_BATCH,
    weight_decay=WEIGHT_DECAY,
    save_total_limit=SAVE_TOTAL_LIM,
    num_train_epochs=NUM_EPOCHS,
    predict_with_generate=True,
    push_to_hub=False,
    logging_dir="./logs",
    logging_steps=1,
)

In [40]:
# Set up trainer
trainer = Seq2SeqTrainer(
    model=peft_model,
    args=training_args,
    train_dataset=tokenized_dataset["train"],
    eval_dataset=tokenized_dataset["test"],
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics
)

C:\Users\pingj\AppData\Local\Temp\ipykernel_17700\3181277492.py:2: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Seq2SeqTrainer.__init__`. Use `processing_class` instead.
  trainer = Seq2SeqTrainer(
No label_names provided for model class `PeftModel`. Since `PeftModel` hides base models input arguments, if label_names is not given, label_names can't be set automatically within `Trainer`. Note that empty label_names list will be used instead.


In [ ]:
trainer.train()

Epoch,Training Loss,Validation Loss,Rouge,Meteor
1,3.038800,2.808671,"{'rouge1': 0.09065554278515837, 'rouge2': 0.020910035534594227, 'rougeL': 0.08894574429163021, 'rougeLsum': 0.08908224130594647}",{'meteor': 0.06441917677899289}
2,2.196300,2.788157,"{'rouge1': 0.09110314795688518, 'rouge2': 0.021867766849236785, 'rougeL': 0.08942928027198242, 'rougeLsum': 0.08960741987696558}",{'meteor': 0.06495903525554858}
3,3.155300,2.776651,"{'rouge1': 0.09366268502913269, 'rouge2': 0.023107689932829748, 'rougeL': 0.09188865313704514, 'rougeLsum': 0.09201423875098197}",{'meteor': 0.06707462212578694}


TrainOutput(global_step=39072, training_loss=3.139088595827567, metrics={'train_runtime': 16121.4385, 'train_samples_per_second': 19.388, 'train_steps_per_second': 2.424, 'total_flos': 9040485033768960.0, 'train_loss': 3.139088595827567, 'epoch': 3.0})

In [48]:
# Trigger the model trainingnatural 512 token length
# trainer.train() 

In [42]:
# from transformers import T5ForConditionalGeneration, T5Tokenizer

last_checkpoint = "./results/checkpoint-39072"

# finetuned_model = T5ForConditionalGeneration.from_pretrained(last_checkpoint)
# tokenizer = T5Tokenizer.from_pretrained(last_checkpoint)

tokenizer = AutoTokenizer.from_pretrained(last_checkpoint)
finetuned_model = AutoModelForSeq2SeqLM.from_pretrained(last_checkpoint)  


In [43]:
my_question = "what is the disadvantage of AI?"
inputs = "Answer this question: " + my_question

inputs = tokenizer(inputs, return_tensors="pt")

In [44]:
outputs = finetuned_model.generate(**inputs)
answer = tokenizer.decode(outputs[0])

print(f"Answer: {answer}")

Answer: <pad> a lack of a human 's intelligence</s>


In [45]:
my_question = "what is the capital of China?"
inputs = "Answer this question: " + my_question

inputs = tokenizer(inputs, return_tensors="pt")

outputs = finetuned_model.generate(**inputs)
answer = tokenizer.decode(outputs[0])

print(f"Answer: {answer}")

Answer: <pad> Beijing</s>


In [27]:
my_question = "where is UK?"
inputs = "Answer this question: " + my_question

inputs = tokenizer(inputs, return_tensors="pt")

outputs = finetuned_model.generate(**inputs)
answer = tokenizer.decode(outputs[0])

print(f"Answer: {answer}")

Answer: <pad> England</s>


In [46]:
my_question = "Who is the most powerful superhero?"
inputs = "Answer this question: " + my_question

inputs = tokenizer(inputs, return_tensors="pt")

outputs = finetuned_model.generate(**inputs)
answer = tokenizer.decode(outputs[0])

print(f"Answer: {answer}")

Answer: <pad> Superman</s>


In [53]:
my_question = "YUGBHKL"
inputs = "Answer this question: " + my_question

inputs = tokenizer(inputs, return_tensors="pt")

outputs = finetuned_model.generate(**inputs)
answer = tokenizer.decode(outputs[0])

print(f"Answer: {answer}")

Answer: <pad> YUGBHKL</s>


In [73]:
# from transformers import T5ForConditionalGeneration, T5Tokenizer

last_checkpoint = "./checkpoint-1803"

# finetuned_model = T5ForConditionalGeneration.from_pretrained(last_checkpoint)
# tokenizer = T5Tokenizer.from_pretrained(last_checkpoint)

tokenizer = AutoTokenizer.from_pretrained(last_checkpoint)
finetuned_model = AutoModelForSeq2SeqLM.from_pretrained(last_checkpoint)  

In [75]:
my_question = "travel plan for UK"
inputs = "Answer this question: " + my_question

inputs = tokenizer(inputs, return_tensors="pt", max_length=512, truncation=True)

outputs = finetuned_model.generate(**inputs)
answer = tokenizer.decode(outputs[0])

print(f"Answer: {answer}")

Answer: <pad> Absolutely! I'd be happy to help you plan a trip to the UK. Here are


In [63]:


my_question = "who is the mother in the how i met your mother?"
inputs = "Answer this question: " + my_question

inputs = tokenizer(inputs, return_tensors="pt")

outputs = finetuned_model.generate(**inputs)
answer = tokenizer.decode(outputs[0])

print(f"Answer: {answer}")

Answer: <pad> Mary</s>


In [3]:
import torch 
torch.version.cuda

'12.4'

In [4]:
import torch
print(torch.__version__)

2.6.0+cu124


In [5]:
import accelerate
import transformers

transformers.__version__, accelerate.__version__

('4.52.4', '1.8.1')